In [1]:
import glob
import os
import time

# from tqdm import tnrange, tqdm_notebook
from tqdm import tqdm

from skimage.transform import resize
from skimage import data, color

import keras.backend as K
from keras.activations import relu
from keras.models import Sequential, Model
from keras.layers import Input,Dense,Reshape, Flatten, Conv2DTranspose, Dropout,BatchNormalization,Activation,PReLU,LeakyReLU,MaxoutDense
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D

from keras.optimizers import Adam,RMSprop
from keras import initializers
from keras.models import load_model

from keras.utils import multi_gpu_model

import numpy as np
import matplotlib.pyplot as plt
import h5py
from sklearn.utils import shuffle


Using TensorFlow backend.


In [3]:
def init_folders(name):
    if not os.path.exists('results'):
        os.mkdir('results')
    if not os.path.exists('results/'+name):
        os.mkdir('results/'+name)
        
def save_loss(name, epoch, Dloss, Gloss):
    path = 'results/'+name+'/loss'
    if not os.path.exists(path):
        os.mkdir(path)
    plt.figure(figsize=(10,8))
    plt.plot(Dloss,label='Dsicriminiative loss')
    plt.plot(Gloss,label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(path+'/loss_%d.png' % epoch)
    
def save_samples(folder, epoch, G,example=16, dim=(10,10),figsize=(10,10), randomDim=100):
    noise = np.random.normal(0,1,size=(example,randomDim))
    generatedImage = G.predict(noise)
    
    plt.figure(figsize=figsize)
    
    for i in range(example):
        plt.subplot(dim[0],dim[1],i+1)
        plt.imshow((.5*generatedImage[i] + .5), interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    
    path = 'results/'+folder+'/samples'
    if not os.path.exists(path):
        os.mkdir(path)
    plt.savefig(path+'/epoch_%d.png' % epoch)
    plt.close()
    
def save_models(name, epoch, d=None, g=None):
    path = 'results/'+name+'/models'
    if not os.path.exists(path):
        os.mkdir(path)
    g.save(path+'/G_%d.h5' % epoch)
    d.save(path+'/D_%d.h5' % epoch)
    
def get_model_memory_usage(batch_size, model):
    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes


# class CustomDataProvider:
#     def __init__(self, batch_size):
#         self.batch_size = batch_size
#         self.file = h5py.File('camelyonpatch_level_2_split_train_x.h5', 'r')
#         self.dataset = self.file['x']
#         self.input_shape = self.dataset[0].shape
#         self.image_number = self.dataset.shape[0]
        
#     def sample(self):
#         random_index = np.random.randint(0, self.image_number - self.batch_size)
#         return (self.dataset[random_index : random_index +  self.batch_size] - 127.5) / 127.5
        
#     def close(self):
#         if self.file:
#             self.file.close()
#         self.file = None
        
#     def __del__(self):
#         self.close()

class PcamDataProvider:
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.file = h5py.File('camelyonpatch_level_2_split_train_x.h5', 'r')
        self.dataset = self.file['x']
        self.input_shape = self.dataset[0].shape
        self.image_number = self.dataset.shape[0]
        
    def sample(self):
        random_index = np.random.randint(0, self.image_number - self.batch_size)
        return (self.dataset[random_index : random_index +  self.batch_size] - 127.5) / 127.5
        
    def close(self):
        if self.file:
            self.file.close()
        self.file = None
        
    def __del__(self):
        self.close()

In [4]:
def wasserstein_loss(y_true,y_pred):
    return K.mean(y_true*y_pred)

### Final critic

In [5]:
def create_critic_final(width, height, channels, kernel_size=3):
    """ Declare discriminator """

    model = Sequential()

    model.add(Conv2D(32, kernel_initializer=initializers.RandomNormal(mean=0,stddev=0.2), 
                     kernel_size=kernel_size, strides=2, 
                     input_shape=(width, height, channels), padding="same"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.15))

    model.add(Conv2D(64, kernel_size=kernel_size, strides=2, padding="same",
             kernel_initializer=initializers.RandomNormal(mean=0,stddev=0.2)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.15))

    model.add(Conv2D(128, kernel_size=kernel_size, strides=2, padding="same",
             kernel_initializer=initializers.RandomNormal(mean=0,stddev=0.2)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.15))

    model.add(Flatten())
    model.add(Dense(256,
             kernel_initializer=initializers.RandomNormal(mean=0,stddev=0.2)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.15))

    model.add(Dense(1,
             kernel_initializer=initializers.RandomNormal(mean=0,stddev=0.2)))

    return model

### Final actor ###

In [6]:
def create_actor_final(width, height, channels, latent_dim=100):
    
    model = Sequential()

    model.add(Dense(32*int(width/4)*int(height/4), input_dim=latent_dim,
                    kernel_initializer=initializers.RandomNormal(mean=0,stddev=0.2)))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))
    model.add(Reshape((int(width/4),int(height/4), 32)))

    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(64, kernel_size=3, strides=1, padding="same"))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))

    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(48, kernel_size=4, strides=2, padding="same"))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))
    
    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(32, kernel_size=3, strides=1, padding="same"))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))

    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(16, kernel_size=4, strides=2, padding="same"))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))
 
    model.add(Conv2D(8, kernel_size=3, strides=1, padding="same"))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))

    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(16, kernel_size=4, strides=2, padding="same"))
#     model.add(BatchNormalization(momentum=0.9, epsilon=0.00002))
    model.add(Activation('relu'))
    
    model.add(Conv2D(channels, kernel_size=3, strides=1, padding="same"))
    model.add(Activation("tanh"))
    
    return model

### Training method ###

In [7]:
def train(name, critic, actor, data,
          batch_size=64,
          epochs=3000, 
          randomDim=100, 
          do_save_loss=True, 
          do_save_models=True, 
          do_save_samples=True, 
          batchCount=1, 
          epoch_offset=0):
    """
    :name: name of the directory, which will store all data
    :critic: model for critic
    :actor: model for critic
    """
    
    init_folders(name)

    batchsize = batch_size

    width, height, channels = 64,64,3

    critic.compile(loss=wasserstein_loss,optimizer=RMSprop(lr=0.00005))
    generator = actor
    discriminator = critic

    discriminator.trainable = False
    gan_input = Input((randomDim,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(gan_input,gan_output)
    gan.compile(loss=wasserstein_loss,optimizer=RMSprop(lr=0.00005))

    Dloss = []
    Gloss = []

#     print('Actor', get_model_memory_usage(batchsize, generator), 'GB')
#     print('Critic', get_model_memory_usage(batchsize*2, discriminator), 'GB')
    
#     print('Epochs',epochs)
#     print('Bathc size',batchsize)
#     print('Batches per epoch',batchCount)
    yGen = -np.ones(batchsize)
    for e in range(epoch_offset+1,epochs+1):
        for idx in tqdm(np.array_split(shuffle(range(data.shape[0])), batchsize), desc="epoch "+str(e)):
            imageBatch = data[idx]

            noise = np.random.normal(0,1,size=[idx.shape[0], randomDim])
            generatedImages = generator.predict(noise)
            
            X_rg = np.concatenate([imageBatch,generatedImages])

            #Train critic
            discriminator.trainable = True
            
            ### Clip weights ###
            weights = [np.clip(w, -0.01, 0.01) for w in discriminator.get_weights()]
            discriminator.set_weights(weights)
            
            yDis = np.ones(2*idx.shape[0])
            yDis[:idx.shape[0]] = -1
            dloss = discriminator.train_on_batch(X_rg, yDis)
            
            #Train actor
            noise = np.random.normal(0,1,size=[batchsize,randomDim])
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise,yGen)

            save_samples(name, e, generator, 16, dim=(4,4))
            save_models(name, e, d=discriminator, g=generator)

#     plt.figure(figsize=(15,6))
#     plt.plot(np.array(Gloss), label='G')
#     plt.plot(np.array(Dloss), label='D')
#     plt.legend()

                

### Start training ###

In [8]:
x_train = (h5py.File('camelyonpatch_level_2_split_train_x.h5', 'r')['x'][:, 16:80,16:80] - 127.5) / 127.5
x_test = (h5py.File('camelyonpatch_level_2_split_test_x.h5', 'r')['x'][:, 16:80,16:80] - 127.5) / 127.5
x_valid = (h5py.File('camelyonpatch_level_2_split_valid_x.h5', 'r')['x'][:, 16:80,16:80] - 127.5) / 127.5
X = np.concatenate([x_train, x_test, x_valid])

X.shape

(327680, 64, 64, 3)

In [9]:
# critic = load_model('results/WGAN_multi_gpu_critic1/models/D_170.h5', custom_objects={'wasserstein_loss': wasserstein_loss})
# actor = load_model('results/WGAN_multi_gpu_critic1/models/G_170.h5', custom_objects={'wasserstein_loss': wasserstein_loss})
critic = create_critic_final(64, 64, 3)
actor = create_actor_final(64, 64, 3, latent_dim=100)

In [10]:
train('WGAN', critic, actor, X, batch_size=64, epochs=1000, randomDim=100)

epoch 1:  36%|███▌      | 23/64 [08:28<14:54, 21.82s/it]

KeyboardInterrupt: 

In [1]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6562814326045976754, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5487336015163307812
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14181360076641362199
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_GPU:1"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16242068446842781647
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_GPU:2"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11027366031515943142
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_GPU:3"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4398014454111972825
 physical_device_desc: "device: XLA_GPU device", name